##**AML competition:**

**match the images in a query set with the images in a much larger set called gallery.**

**ResNet50 versions:**

this code has two parts: in the first part we check how the pre-trained model works, we extract the features and try to match images from the query to the gallery images. 
After this initial exploration, in the second and most important part, we fit the model, adding to it some layers, and produce a dictionary of matching images.

**FIRST PART**

Import the necessary libraries.

In [1]:
import numpy as np
from numpy.linalg import norm
from tqdm import tqdm, tqdm_notebook
import os
import cv2
import random
import time
import math
import tensorflow
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
import keras
import PIL
from PIL import Image
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
%tensorflow_version 1.x
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
import time
from sklearn.decomposition import PCA
import random
from scipy.spatial import distance

TensorFlow is already loaded. Please restart the runtime to change versions.


Use the colab GPU for faster execution of the code.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Connect colab to google drive, where the dataset with all the images is saved.

In [ ]:

!unzip /content/drive/MyDrive/challenge_test_data.zip


In [4]:
GALLERY_PATH = "/content/gallery"
QUERY_PATH = "/content/query"

In [5]:
SAVE_MODEL_PATH = '/content/drive/MyDrive/models/saved_models/ResNet50.h5'
SAVE_MODEL_WEIGHTS_PATH = '/content/drive/MyDrive/models/saved_weights/ResNet50.h5'


In [ ]:
model=tf.keras.applications.ResNet50(weights='imagenet',
                         include_top=False, #does not include a fully-connected layer at the top of the network
                         input_shape=(224, 224, 3)) #global max pooling

In [7]:
new_model = Sequential()
new_model.add(model)
new_model.add(Flatten())

In [8]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [9]:
feat_extractor = Model(inputs=new_model.input, outputs=new_model.get_layer("flatten").output)
feat_extractor.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50_input (InputLayer)  [(None, 224, 224, 3)]    0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
import utils
gallery = utils.Dataset(data_path=GALLERY_PATH).get_dataset()
query = utils.Dataset(data_path=QUERY_PATH).get_dataset()

gallery_features, gallery_urls, gallery_labels = utils.compute_features(gallery, feat_extractor)
query_features, query_urls, query_labels = utils.compute_features(query, feat_extractor)

results_eu = utils.compute_results(query_features, gallery_features, query_urls, gallery_urls)
results_cos = utils.compute_results(query_features, gallery_features, query_urls, gallery_urls, dist= 'cosine')

# Competition

In [ ]:
import requests
import json


def submit(results, url="http://tinyurl.com/IML2022"):
    res = json.dumps(results)
    response = requests.post(url, res)
    try:
        result = json.loads(response.text)
        print(f"accuracy is {result['results']}")
    except json.JSONDecodeError:
        print(f"ERROR: {response.text}")



mydata = dict()
mydata['groupname'] = "The Ostriches"

mydata["images"] = results_eu
submit(mydata)

mydata["images"] = results_cos
submit(mydata)